In [1]:
import json
import requests
import pandas as pd
import geopandas as gpd

# Problem 3: Populations

For this part, you'll need to request a census API key. Using the 2019 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. Merge this new data with the burglaries data above.

In [2]:
with open('C:/Users/jrior/Documents/NSS/api_keys.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['census_data']

In [3]:
endpoint = "https://api.census.gov/data/2019/acs/acs5?"
query = f"get=NAME,B01001_001E&for=tract:*&in=state:47"#&key={api_key}"
response2 = requests.get(endpoint+query)
response2

<Response [200]>

In [4]:
res2_dict = response2.json()
tn_pops = pd.DataFrame().from_dict(res2_dict)
tn_pops.columns = tn_pops.loc[0]
tn_pops = (tn_pops.drop(tn_pops.index[0])
           .reset_index(drop = True)
           .rename(columns = {'NAME': 'name',
                              'B01001_001E': 'population'}
                  )
           .astype({'population':'int'})
          )

In [5]:
endpoint = "https://api.census.gov/data/2019/acs/acs5/subject?"
query = f"get=NAME,S1901_C01_012E&for=tract:*&in=state:47"#&key={api_key}"
response3 = requests.get(endpoint+query)
response3

<Response [200]>

In [6]:
res3_dict = response3.json()
tn_med_inc = pd.DataFrame().from_dict(res3_dict)
tn_med_inc.columns = tn_med_inc.loc[0]
tn_med_inc = (tn_med_inc.drop(tn_med_inc.index[0])
              .reset_index(drop = True)
              .rename(columns = {'NAME': 'name',
                                 'S1901_C01_012E': 'median_income'}
                     )
              .astype({'median_income':'int'})
             )

In [7]:
endpoint = "https://api.census.gov/data/2019/acs/acs5/?"
query = f"get=NAME,B19083_001E&for=tract:*&in=state:47"#&key={api_key}"
response4 = requests.get(endpoint+query)
response4
res4_dict = response4.json()
tn_gini = pd.DataFrame().from_dict(res4_dict)
tn_gini.columns = tn_gini.loc[0]
tn_gini = (tn_gini.drop(tn_gini.index[0])
              .reset_index(drop = True)
              .rename(columns = {'NAME': 'name',
                                 'B19083_001E': 'gini_index'}
                     )
              .astype({'gini_index':'float'})
             )

In [8]:
tn_census = (tn_pops.merge(tn_med_inc)
                   .merge(tn_gini)
            )

tn_census[['county_name', 'state_name']] = (tn_census['name'].str.split(pat = ', ',
                                         expand = True
                                        )
                                       .drop(columns = [0])
    )
tn_census = tn_census.drop(columns = 'name')

In [9]:
tn_burglary = gpd.read_file("../data/tn_burglary.geojson")

In [10]:
tn = tn_census.merge(tn_burglary)
tn = gpd.GeoDataFrame(tn)

In [11]:
tn.to_file("../data/tn.geojson", index = False, driver='GeoJSON')